In [2]:
%run SetUp.ipynb

In [3]:
angles_from_vqe = np.array([ 0.0310549 ,  0.08593939, -0.0328307 , -0.18265392,  2.99830422,
       -0.02271861])

# Classical Calculation

In [16]:

# Generate Fake Vacuum
fake_vac = np.ndarray.flatten(ft.reduce(sp.sparse.kron, [[1,0]]* nq).toarray())


In [ ]:

# Turn on 3 links in the middle
def activate(init):
    if nq <= 5: raise Exception (" Size of system too small. Operate on number of qubits >= 6")
    loc = (nq-5) // 2

    act_mat = pad_op(loc, nq, sp.linalg.expm(ft.reduce(sp.sparse.krom,[proj0,x,x,x,proj0])),5) # needs change
    
    return act_mat.dot(init)
    
# Unitary evolution on any initial with angles found in VQE1 to establish true vaccum
def true_vac(init):
    vqe_form(angles_from_vqe).dot(fake_vac)

# Initialize State
initial = true_vac(activate(fake_vac))

In [ ]:
# Time evolution. Evolve with e^(-iHt)

t = 5
final_state = sp.linalg.expm(-1j * Ham * t).dot(initial)

In [ ]:
# Plot State to check evolution results. Want to evolve to maximize using the available classical qubits 
plt.plot(list(range(2**nq)),initial,label = 'Initial')
plt.plot(list(range(2**nq)),final_state,label = 'Final')

# Move to QC

In [ ]:
nQ = 100

In [ ]:
# VQE2 to learn how to prepare the state generated above on the smaller number of qubits

In [ ]:
# Use VQE1 results to prepare the 100 qubit QC to vacuum state

full_cir = one_layer(nQ, angles_from_vqe)

testcir = Statevector.from_int(0, 2**nQ)
testcir = testcir.evolve(test)
#results = testcir.probabilities_dict()
results = np.real(testcir.data)

plt.plot(list(range(2**nQ)), results,'b.',label = 'VQE 1 layer')

In [ ]:
# Put the smaller evolved state on to QC

In [ ]:
# Time evolution to the extend of QC available depth
# e^iHt needs to be redefined in gates form and Trotterized 